# Vision guided navigation with Thymio - Final project
Authors: Ahmed Boubakry, Charles de Fournas, Julie Favre, Paul Richard

## 1. Introduction

The aim of this project is to have a Thymio robot moving on a plane with global and local naviagation. It should find the best path to the finish goal, while navigating to avoid obstacles on its way. The algorithms and methods we are implementing are:
- Computer Vision in order to create the map and retrieve the robot's position
- A* algorithm to find the optimal path
- Local navigation to avoid physical objects
- Filtering with Extended Kalman Filter to estimate accuretly the position
- Controller to correct the trajectory after every move

## 2. Environment of the project

We have opted for a black and white grid as our environment, where white cells signify walkable areas and black cells represent walls. The robot's objective is to determine the optimal path from its starting position to a designated goal cell, indicated in blue.

For visualization purposes, we print the grid on an A0 page with 8 columns and 6 rows. The page's borders serve as reference points for the camera to deduce the grid. We've outlined the grid using black tape, aiding in its detection.

Obstacles are depicted by black cartoon representations, and the goal is visually distinguished by a blue cartoon. Additionally, on the robot, we've strategically positioned two black points – a smaller one in front of a larger one – to help the camera ascertain the robot's orientation.

To enhance the camera's visibility, we've strategically placed multiple lamps to illuminate the area. These lamps are intended to provide ample lighting for improved camera performance.

In consideration of the Thymio sensor, obstacles are designed to be solid, ensuring effective detection by the sensor. It's important to note that these obstacles are shaped differently from rounds, as our camera primarily detects round forms for our robot.

## 3. Computer vision

code pour le cv et tout ce qu'on lit de la cam
expliquer tout ce qu'on fait !
ajouter image du décalage des centres
ajouter image de cv (ou on voit robot et cadrage)

We noticed that when the robot was on the edge of the map, the robot was shifted towards the center of the map, due to the angle with the camera. (see image) To correct this, we added a distortion correction array, that computes the virtual center of the cell from the camera point of vue.




## 4. Path finding

This module is very dependant on the vision module, since the vision module maps the area. The vision module returns a state matrix, giving us the information on which square is occupied or not. After manipulating this matrix so it can work conviniently with the function that runs with the A* function, we can run the algorithm. The shortest path is then given to us in the form of a double array, called in the code `path`.

We have chosen the A* algorithm because our sceario, which is a robot trying to find the goal in a 'maze', corresponds exactly on how we want the shortest path be computed from. Using the heuristic function h wasn't really necessary since all the distances the robot runs have an equal distance from each other. But in the case where our robot can use 8 directions paths, it would've helped in fiding the shortest path.

Once `path` is filled, we run a loop that will make the robot move to each location one by one, starting by loading the values of its current position and the next one : 

            for i in range(len(path)-1):
                x1, y1 = path[i]
                x2, y2 = path[i + 1]


We make the robot move three times to move from a location to another, so it has the time between each movement to check the obstacles, and update the `obstacle_state` variable. If the robot encounters an obstacle, it will run another loop that makes the robot avoid the obstacle (see Local Navigation section).

(insert the prints here)

All the movements that the robot makes are computed via mathematical functions, (namely, `calculate_angle_degrees` and `calculate_distance`), and given to `move_robot` and `rotate_robot`, who will give a command to a motor and put to sleep using `time.sleep()` during the time the command need to be applied.

Then, we check if the robot has reached its goal by looking at the robot position and the goal position via the camera. If the two positions matches (with a margin error around 50 pixels), then we stop the program.

## 5. Local Navigation

### Theory : 

To ensure obstacle avoidance, we have opted for a convinient way to do so by using the Artificial Neural Network method. By using the five sensors in front of the Thymio, the robot can modify the speed of its wheel depending on the value returned by the thymio. We can use the following equation to compute the speed sent to the motors :

\begin{array}{rcl}
v_{Left} = w_{1l}x_1 + w_{2l}x_2 + w_{3l}x_3 + w_{4l}x_4 + w_{5l}x_5 \\
v_{Right} = w_{1r}x_1 + w_{2r}x_2 + w_{3r}x_3 + w_{4r}x_4 + w_{5r}x_5
\end{array}

Thanks to the ANN method, ach proximity sensor has one weight coefficient. The coefficients $x_1, \cdots, x_5$ are used as the measurements of the horizontal distances that the robot can sense.

#### Obstacle detection using ANN : 

    speed0 = 50      
    SpeedGainObst = [6, 4, -2, -6, -8]
    SpeedLeftWheel = speed0
    SpeedRightWheel = speed0

    for i in range(5):
        speedLeftWheel += obst[i] * SpeedGainObst[i] // 100
        speedRightWheel += obst[i] * SpeedGainObst[4 - i] // 100

    return speedLeftWheel, speedRightWheel

### How to find the weights? 

In our code, we used the following weights : $[w_{1l}, w_{2l}, w_{3l}, w_{4l}, w_{5l}]=[0.06,0.04,-0.02,-0.06,-0.08]$ and $[w_{1r}, w_{2r}, w_{3r}, w_{4r}, w_{5r}]=[-0.08,-0.06,-0.02,0.04,0.06]$, by using a nominal speed of 100, and all of this by testing and correcting our values.  

These coefficients are used so that the obstacle is dodged in a reasonnable manner (we want to stay in the velocity bound, and not give an angle which is too tight or too wide).

### Switching from local navigation to global navigation : 

In order to be able to return in the main state after making the global navigation, we use a function that checks the robot state (state_robot). When all the sensor do not detect something in front of them, and has been detecting obstacles before, the state becomes 2. When the case is the same but the sensor has never been detecting something before, the state remains 1. And finally, when the sensor detects and obstacle, we enter in the "obstacle avoidance" mode, which is described by the state 0.

### Kidnapping 

The objective of the kidnapping prevention mechanism for the Thymio robot is to detect and respond to instances where the robot is lifted off the ground, signaling a potential kidnapping scenario. This is achieved through continuous monitoring of the ground sensor, which detects changes in the robot's contact with the ground surface.

The Thymio robot is equipped with a ground sensor that provides real-time information about the contact between the robot and the ground. The detection mechanism relies on regular checks of the ground sensor readings during the robot's movement. The code continuously monitors the sensor output to ensure that the robot is in contact with the ground. If a deviation from the expected sensor values occurs, indicating that the robot is no longer on the ground, the system interprets this as a potential kidnapping event.

Upon detecting that the robot is not on the ground, the code initiates a predefined response strategy. The chosen approach involves resetting the robot's state and restarting the code from the beginning. We use the variable `was_kidnapped` to ensure that we can distinguish the case where the robot needs to restart or not.

## 6. Filtering

To be sure that we have the correct position of the robot even when the camera fails, we implemented a filtering feature. As our model was not linear, we went with the Extended Kalman Filter. This filter takes as input the position measured by the camera, and the instaneous speed of the robot. Every time it is called, it updates the position based on the previous position, the camera input, and the model of the robot (where it is supposed to be given the speed and previous position).

We call the filter after every movement, giving the time period and the speed, thanks to the method `filter_call()` below.

The rest of the code is all in ``ExtendedKalmanFilter.py``

In [ ]:
ekf = EKF(1, initial_pos, initial_speed) 

## 7. Controller

For the motion control, we made use of a controller. After every cell the robot moved to, we check if its in the expected position. If the distance between the center of the robot and the center of the cell it is supposed to be in is too large, we execute the controller in two step. If the distance is good but the orientation of the robot is too different of the expected orientation, we execute the controller in one step. If it is good enough, we continue the path without interfering.
The controller in three steps:
- rotate the robot in direction of the center of the cell
- move the robot towards the center
- rotate the robot in the expected orientation to continue the path

The controller in one step:
- rotate the robot in the expected orientation


## 8. Project run

main loop !
blablater sur les choix de synchrone vs asynchrone et quelles methodes on appelle quand


## 9. Conclusion

In conclusion, we managed to implement a moving robot on the grid. The few drawbacks we have are:

### 10. Sources

ChatGPT to help with debugging

filter: https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
